In [5]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import gridforceplugin
import sys

In [2]:
pdb = app.PDBFile('input.pdb')
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer, constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.context.set
simulation.reporters.append(app.PDBReporter('output.pdb', 1000))
simulation.reporters.append(app.StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-142724.37343246874,292.7062333736618
2000,-140901.49843246874,298.56603288209357
3000,-141074.43593246874,303.264613868362
4000,-141151.93593246874,303.96825718821157
5000,-140619.18593246874,293.31781654068425
6000,-141213.77968246874,296.8305322078249
7000,-140459.99843246874,300.8044714781755
8000,-140261.37343246874,299.4970856494876
9000,-140109.43593246874,297.74856739730467
10000,-140441.12343246874,302.3177990539783


In [8]:
top = app.topology.Topology()
top.addAtom
# chain = top.addChain(id=0)
# res1 = top.addResidue("C1", chain)
# top.addAtom("C1", app.element.carbon, res1)
# t, r = forcefield.generateTemplatesForUnmatchedResidues(top)
system = forcefield.createSystem(top, nonbondedMethod=app.NoCutoff, nonbondedCutoff=1*unit.nanometer, constraints=app.HBonds)
print(top.getNumAtoms())
print(system.getNumParticles())
simulation = app.Simulation(top, system, integrator)
simulation.system.addParticle(12.)
simulation.context.setPositions(top.positions)
simulation.minimizeEnergy()
simulation.reporters.append(app.PDBReporter('output.pdb', 1000))
simulation.reporters.append(app.StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))
simulation.step(10000)

0
0


OpenMMException: Cannot create a Context for a System with no particles

In [9]:
# =============================================================================
# Specify simulation parameters
# =============================================================================

nparticles = 2 # number of particles

mass = 39.9 * unit.amu # mass
sigma = 3.4 * unit.angstrom # Lennard-Jones sigma
epsilon = 0.238 * unit.kilocalories_per_mole # Lennard-Jones well-depth
charge = 0.0 * unit.elementary_charge # argon model has no charge

nequil_steps = 5000 # number of dynamics steps for equilibration
nprod_steps = 500 # number of dynamics steps per production iteration
nprod_iterations = 50 # number of production iterations per lambda value

reduced_density = 0.85 # reduced density rho*sigma^3
temperature = 300 * unit.kelvin # temperature
pressure = 1 * unit.atmosphere # pressure
collision_rate = 5.0 / unit.picosecond # collision rate for Langevin thermostat
timestep = 1 * unit.femtosecond # integrator timestep

lambda_values = [0.0, 0.25, 0.5, 0.75, 1.0]
nlambda = len(lambda_values)

In [10]:
# =============================================================================
# Compute box size.
# =============================================================================

volume = nparticles*(sigma**3)/reduced_density
box_edge = volume**(1.0/3.0)
cutoff = min(box_edge*0.49, 2.5*sigma) # Compute cutoff
print("sigma = %s" % sigma)
print("box_edge = %s" % box_edge)
print("cutoff = %s" % cutoff)

sigma = 3.4 A
box_edge = 21.53561071635121 A
cutoff = 8.5 A


In [29]:
# =============================================================================
# Build systems at each alchemical lambda value.
# =============================================================================

print("Building alchemically-modified systems...")
alchemical_systems = list() # alchemical_systems[i] is the alchemically-modified System object corresponding to lambda_values[i]
for lambda_value in lambda_values:
    # Create argon system where first particle is alchemically modified by lambda_value.
    system = mm.System() 
    system.setDefaultPeriodicBoxVectors(mm.Vec3(box_edge, 0, 0), mm.Vec3(0, box_edge, 0), mm.Vec3(0, 0, box_edge))
    force = mm.CustomNonbondedForce("""4*epsilon*l12*(1/((alphaLJ*(1-l12) + (r/sigma)^6)^2) - 1/(alphaLJ*(1-l12) + (r/sigma)^6));
                                   sigma=0.5*(sigma1+sigma2);
                                   epsilon=sqrt(epsilon1*epsilon2);
                                   alphaLJ=0.5;
                                   l12=1-(1-lambda)*step(useLambda1+useLambda2-0.5)""")
    force.addPerParticleParameter("sigma")
    force.addPerParticleParameter("epsilon")
    force.addPerParticleParameter("useLambda")
    force.addGlobalParameter("lambda", lambda_value)
    force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
    force.setCutoffDistance(cutoff) 
    for particle_index in range(nparticles):
        system.addParticle(mass)
        if (particle_index == 0):
            # Add alchemically-modified particle.
            force.addParticle([sigma, epsilon, 1])
        else:
            # Add normal particle.
            force.addParticle([sigma, epsilon, 0])
    system.addForce(force)

   # Add barostat.
   #barostat = MonteCarloBarostat(pressure, temperature)
   #system.addForce(barostat)

   # Store system.
    alchemical_systems.append(system)

# Create random initial positions.
import numpy.random
positions = unit.Quantity(numpy.random.uniform(high=box_edge/unit.angstroms, size=[nparticles,3]), unit.angstrom)

Building alchemically-modified systems...


In [ ]:
# =============================================================================
# Run simulations at each alchemical lambda value.
# Reduced potentials of sampled configurations are computed for all alchemical states for use with MBAR analysis.
# =============================================================================

u_kln = numpy.zeros([nlambda, nlambda, nprod_iterations], numpy.float64) # u_kln[k,l,n] is reduced potential of snapshot n from simulation k at alchemical state l
for (lambda_index, lambda_value) in enumerate(lambda_values):
    # Create Integrator and Context.
    integrator = mm.LangevinIntegrator(temperature, collision_rate, timestep)
    context = mm.Context(alchemical_systems[lambda_index], integrator)

    # Initiate from last set of positions.
    context.setPositions(positions)

    # Minimize energy from coordinates.
    print("Lambda %d/%d : minimizing..." % (lambda_index+1, nlambda))
    mm.LocalEnergyMinimizer.minimize(context)

    # Equilibrate.
    print("Lambda %d/%d : equilibrating..." % (lambda_index+1, nlambda))
    integrator.step(nequil_steps)

    # Sample.
    position_history = list() # position_history[i] is the set of positions after iteration i
    for iteration in range(nprod_iterations):
        print("Lambda %d/%d : production iteration %d/%d" % (lambda_index+1, nlambda, iteration+1, nprod_iterations))

        # Run dynamics.
        integrator.step(nprod_steps)

        # Get coordinates.
        state = context.getState(getPositions=True)
        positions = state.getPositions(asNumpy=True)

        # Store positions.
        position_history.append(positions)
      
    # Clean up.
    del context, integrator

    # Compute reduced potentials of all snapshots at all alchemical states for MBAR.
    print("Lambda %d/%d : computing energies at all states..." % (lambda_index+1, nlambda))
    beta = 1.0 / (unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA * temperature) # inverse temperature
    for l in range(nlambda):
        # Set up Context just to evaluate energies.
        integrator = mm.VerletIntegrator(timestep)
        context = mm.Context(alchemical_systems[l], integrator)

        # Compute reduced potentials of all snapshots.
        for n in range(nprod_iterations):
         context.setPositions(position_history[n])
         state = context.getState(getEnergy=True)
         u_kln[lambda_index, l, n] = beta * state.getPotentialEnergy()

        # Clean up.
        del context, integrator

In [7]:
# =============================================================================
# Specify simulation parameters
# =============================================================================

nparticles = 2 # number of particles

mass = 12. * unit.amu # mass
sigma = 3.4 * unit.angstrom # Lennard-Jones sigma
epsilon = 0.238 * unit.kilocalories_per_mole # Lennard-Jones well-depth
charge1 = 1. * unit.elementary_charge # argon model has no charge
charge2 = -0.2 * unit.elementary_charge # argon model has no charge

nequil_steps = 5000 # number of dynamics steps for equilibration
nprod_steps = 500 # number of dynamics steps per production iteration
nprod_iterations = 50 # number of production iterations per lambda value

reduced_density = 0.85 # reduced density rho*sigma^3
temperature = 300 * unit.kelvin # temperature
pressure = 1 * unit.atmosphere # pressure
collision_rate = 5.0 / unit.picosecond # collision rate for Langevin thermostat
timestep = 1 * unit.femtosecond # integrator timestep

lambda_values = [0.0, 0.25, 0.5, 0.75, 1.0]
nlambda = len(lambda_values)

# =============================================================================
# Compute box size.
# =============================================================================

volume = nparticles*(sigma**3)/reduced_density
box_edge = volume**(1.0/3.0)
cutoff = min(box_edge*0.49, 2.5*sigma) # Compute cutoff
print("sigma = %s" % sigma)
print("box_edge = %s" % box_edge)
print("cutoff = %s" % cutoff)

sigma = 3.4 A
box_edge = 4.522194877312084 A
cutoff = 2.2158754898829214 A


In [8]:
# =============================================================================
# Build simple system to test AlGDock
# =============================================================================

system = mm.System() 
system.setDefaultPeriodicBoxVectors(mm.Vec3(box_edge, 0, 0), mm.Vec3(0, box_edge, 0), mm.Vec3(0, 0, box_edge))
force = gridforceplugin.GridForce()
force = mm.CustomNonbondedForce()

In [4]:
force.addPerParticleParameter("sigma")
force.addPerParticleParameter("epsilon")
force.addPerParticleParameter("useLambda")
force.addGlobalParameter("lambda", lambda_value)
force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
force.setCutoffDistance(cutoff) 
for particle_index in range(nparticles):
    system.addParticle(mass)
    if (particle_index == 0):
        # Add alchemically-modified particle.
        force.addParticle([sigma, epsilon, 1])
    else:
        # Add normal particle.
        force.addParticle([sigma, epsilon, 0])
system.addForce(force)